In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import os 
from pylab import *

**Limites da região**\
Benguela: Latitude: 39°S - 17°S; Longitude: 8°E to 31°E\

In [ ]:
benguela=xr.open_dataset('/home/lipfp/Documentos/Marine_Heatwaves/saidas/Benguela/SST_MHW_Benguela_Agulhas.nc', chunks={'time':10})

### para reagrupar os chunks 
### benguela.unify_chunks()

In [ ]:
time=benguela.time
newtime=[datetime.datetime.fromordinal(int(time[i].data)) for i in range(len(time))] # criando o intervalo de tempo
benguela=benguela.assign(time=newtime)

del time

In [ ]:
benguela=benguela.sel(time=slice('1990','2020'))
benguela=benguela.where(benguela>-1000)

In [ ]:
mhw_category = benguela.mhw_category.where(benguela.mhw_category>0)
sst_anom=benguela.sst_anom #sst anomalia
sst=benguela.sst      #sst 
seas=benguela.seas    #climatologia
thresh=benguela.thresh #percentil 90
sst_anom_event=benguela.sst_anom.where(benguela.mhw_category>0)
sst_acum=sst_anom_event.where(sst_anom_event>0)

##Transformando a variável "mhw_category" para a contagem.
mhw_category_new = mhw_category*0
mhw_category_new = mhw_category_new+1

In [ ]:
land = cfeature.NaturalEarthFeature('physical', 
                                    'land', 
                                    '50m', 
                                    edgecolor='black',
                                    facecolor='gray')
fig, axis = plt.subplots(
    1, 1, subplot_kw=dict(projection=ccrs.PlateCarree())
)
### somatório dos valores positivos 
sst_acum.sum(dim='time').plot.contourf(
    ax=axis,
    transform=ccrs.PlateCarree(),  
    cmap='hot_r',
    cbar_kwargs={"orientation": "horizontal", "label":"°C-days","shrink": 0.8},
    robust=True, levels=15
)
# plt.text( 349, -6, 'South Atlantic',fontdict={'fontsize':'12'},horizontalalignment='right', transform=ccrs.Geodetic())
# plt.text( 319, 41, 'North Atlantic',fontdict={'fontsize':'12'},horizontalalignment='right', transform=ccrs.Geodetic())
axis.coastlines() 
axis.add_feature(land, linewidth=0.25)
axis.add_feature(cfeature.BORDERS, linewidth=1)
gl=axis.gridlines(draw_labels=True,linestyle='--')
gl.top_labels=False
gl.right_labels
gl.left_labels=False
gl.bottom_labels
plt.title('MHW Intensity (1990-2020)', size=15)
fig.set_size_inches(10,10)
# fig.savefig('mhw_intensity.jpeg')

In [ ]:
## gráfico mostrando a quantidade de dias com MHW
land = cfeature.NaturalEarthFeature('physical', 
                                    'land', 
                                    '50m', 
                                    edgecolor='black',
                                    facecolor='gray')
fig, axis = plt.subplots(
    1, 1, subplot_kw=dict(projection=ccrs.PlateCarree())
)

mhw_category_new.sum(dim='time').plot.contourf(vmin=500,vmax=1000,
    ax=axis,
    transform=ccrs.PlateCarree(), 
    cmap='RdYlBu_r',
    cbar_kwargs={"orientation": "horizontal", "label":"Nº of MHW days | 1982-1985 ","shrink": 0.8},
    robust=True,levels=15
)
# plt.text( 349, - 6, 'South Atlantic',fontdict={'fontsize':'12'},horizontalalignment='right', transform=ccrs.Geodetic())
# plt.text( 319, 41, 'North Atlantic',fontdict={'fontsize':'12'},horizontalalignment='right', transform=ccrs.Geodetic())
axis.coastlines() 
axis.add_feature(land, linewidth=0.25)
axis.add_feature(cfeature.BORDERS, linewidth=1)
gl=axis.gridlines(draw_labels=True,linestyle='--')
gl.top_labels=False
gl.right_labels
gl.left_labels=False
gl.bottom_labels
plt.title('MHW frequency  (1990-2020)', size=15)
fig.set_size_inches(10,10)
# fig.savefig('MHW_frequency.jpeg')

**Selecionando a região para trabalhar nos gráficos**

In [ ]:
## anomalia media de toda a série temporal
land = cfeature.NaturalEarthFeature('physical', 
                                    'land', 
                                    '50m', 
                                    edgecolor='black',
                                    facecolor='gray')
fig, axis = plt.subplots(
    1, 1, subplot_kw=dict(projection=ccrs.PlateCarree())
)

sst_anom.mean(dim='time').plot.contourf(
    ax=axis,
    transform=ccrs.PlateCarree(), 
    cmap='Spectral_r',
    cbar_kwargs={"orientation": "horizontal", "label":"°C","shrink": 0.8},
    robust=True,levels=15
)
# plt.text( 349, - 6, 'South Atlantic',fontdict={'fontsize':'12'},horizontalalignment='right', transform=ccrs.Geodetic())
# plt.text( 319, 41, 'North Atlantic',fontdict={'fontsize':'12'},horizontalalignment='right', transform=ccrs.Geodetic())
# plt.plot([9,11.7],[-17.8,-17.8],color='k', lw=2, ls='--')
plt.plot([10.9,10.9],[-17.8,-17.8],color='k',  marker='o',lw=6, linestyle='dashed',markersize=12)
axis.coastlines() 
axis.add_feature(land, linewidth=0.25)
axis.add_feature(cfeature.BORDERS, linewidth=1)
gl=axis.gridlines(draw_labels=True,linestyle='--')
gl.top_labels=False
gl.right_labels
gl.left_labels=False
gl.bottom_labels
plt.title('SST anomaly | (1990-2020)', size=15)
fig.set_size_inches(10,10)
# fig.savefig('SST_anomaly.jpeg')

**SST-CLIMATOLOGIA- PERCENTIL 90**

In [ ]:
lat18=benguela.sel(lat=[-17],lon=[10.9], method='nearest')
lat18_res=lat18.resample(time='11d').mean()

In [ ]:
fig, ax=plt.subplots(figsize=(40,10))

a=lat18_res.sst.mean(dim='lon').plot(lw=2)
b=lat18_res.seas.mean(dim='lon').plot()
c=lat18_res.thresh.mean(dim='lon').plot(lw=2, color='g')
d=plt.axhline(y = 25, color = 'r', linestyle = '-')
plt.legend(loc=3,labels=('SST','SST mean','pc 90','SST max'), prop={'size': 16})
plt.title('Latitude ~17.8 S', size=20)
plt.xticks(size=16)
plt.yticks(size=16)
plt.ylabel('SST', size=16)
plt.xlabel('Time', size=16)
plt.xlim('1989','2021')
ax.xaxis.set_major_locator(plt.MaxNLocator(20))

fill([9000,9000,9400,9400], [15,27,27,15], 'r', alpha=0.1)
fill([14800,14800,15200,15200], [15,27,27,15], 'r', alpha=0.1)
fill([18100,18100,18500,18500], [15,27,27,15], 'r', alpha=0.1)

plt.savefig('lat17_allseries.jpeg')

In [ ]:
benguela.sst.isel(time=1995).sel(lat=[-18, -25, -30], method="nearest").plot(
    x="lon", hue="lat")